# Prosaccade Feedback Session Analysis

This notebook analyzes data from a saccade task where:
- The animal's eye movements map to a green dot on the monitor
- A blue target dot appears at some location
- The animal is rewarded when the green dot (eye position) touches the blue dot (target)
- After a delay, a new trial starts

The notebook produces:
1. Trajectory plots showing eye position paths relative to target position
2. Time-to-target analysis showing trial durations
3. Various statistical analyses and comparisons

## 1. Setup and Imports

In [ ]:
from __future__ import annotations

import sys
from pathlib import Path
import re

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Enable inline plotting
%matplotlib inline

# Add the Python folder to path so we can import from analysis module
notebook_dir = Path.cwd()
python_dir = notebook_dir.parent
if str(python_dir) not in sys.path:
    sys.path.insert(0, str(python_dir))

# Import functions from the existing analysis script
from analysis.prosaccade_feedback_session import (
    load_feedback_data,
    identify_and_filter_failed_trials,
    extract_trial_trajectories,
    plot_trajectories,
    plot_trajectories_by_direction,
    plot_density_heatmap,
    plot_time_to_target,
    plot_path_length,
    plot_final_positions_by_target,
    interactive_fixation_viewer,
    compare_left_right_performance,
    compare_visible_invisible_performance,
    FIXATION_MIN_DURATION,
    FIXATION_MAX_MOVEMENT,
)

print(f"Notebook directory: {notebook_dir}")
print(f"Python directory: {python_dir}")
print(f"Default fixation parameters: min_duration={FIXATION_MIN_DURATION}s, max_movement={FIXATION_MAX_MOVEMENT}")
print("Setup complete!")

## 2. Configuration

Set your data folder path and parameters here:

In [ ]:
# =============================================================================
# CONFIGURATION - MODIFY THESE VALUES
# =============================================================================

# Path to your data folder containing the CSV files
# Use raw string (r"...") for Windows paths, or forward slashes
FOLDER_PATH = Path(r"C:\path\to\your\data\folder")

# Animal ID
ANIMAL_ID = "Tsh001"

# Results directory (set to None to use FOLDER_PATH/results)
RESULTS_DIR = None

# Trial duration filters for analysis
TRIAL_MIN_DURATION = 0.01  # seconds
TRIAL_MAX_DURATION = 15.0  # seconds

# Whether to include failed trials in analysis
INCLUDE_FAILED_TRIALS = False

# =============================================================================

# Set up results directory
if RESULTS_DIR is None:
    RESULTS_DIR = FOLDER_PATH / "results"
else:
    RESULTS_DIR = Path(RESULTS_DIR)

# Try to extract date from folder name
date_match = re.search(r'\d{4}-\d{2}-\d{2}', str(FOLDER_PATH))
DATE_STR = date_match.group() if date_match else ""

print(f"Data folder: {FOLDER_PATH}")
print(f"Results directory: {RESULTS_DIR}")
print(f"Animal ID: {ANIMAL_ID}")
print(f"Session date: {DATE_STR}")

## 3. Load Data

In [ ]:
# Load the three CSV files
eot_df, eye_df, target_df_all = load_feedback_data(FOLDER_PATH, ANIMAL_ID)

# Add original trial numbers
target_df_all['original_trial_number'] = range(1, len(target_df_all) + 1)

In [ ]:
# Identify and filter failed trials
target_df_successful, failed_indices, successful_indices = identify_and_filter_failed_trials(
    target_df_all, eot_df, exclude_failed=True
)

In [ ]:
# Extract trial trajectories
print("Extracting all trials...")
trials_all = extract_trial_trajectories(eot_df, eye_df, target_df_all,
                                        successful_indices=successful_indices)

# Separate successful trials
trials_successful = [t for t in trials_all if not t.get('trial_failed', False) and t.get('has_eye_data', True)]

# Select which trials to use for analysis
if INCLUDE_FAILED_TRIALS:
    trials_for_analysis = trials_all
    print(f"\nUsing ALL {len(trials_all)} trials for analysis")
else:
    trials_for_analysis = trials_successful
    print(f"\nUsing {len(trials_successful)} successful trials for analysis")

## 4. Trajectory Visualizations

In [ ]:
# Plot all trajectories (colored by trial number)
fig_traj = plot_trajectories(trials_for_analysis, RESULTS_DIR, ANIMAL_ID, DATE_STR)
plt.show()

In [ ]:
# Plot trajectories by direction (left vs right targets)
fig_dir = plot_trajectories_by_direction(trials_for_analysis, RESULTS_DIR, ANIMAL_ID, DATE_STR)
plt.show()

In [ ]:
# Plot density heatmap
fig_heat = plot_density_heatmap(trials_for_analysis, RESULTS_DIR, ANIMAL_ID, DATE_STR)
plt.show()

## 5. Performance Metrics

In [ ]:
# Plot time to target across trials
fig_time = plot_time_to_target(trials_for_analysis, RESULTS_DIR, ANIMAL_ID, DATE_STR)
plt.show()

In [ ]:
# Plot path length across trials
fig_path = plot_path_length(trials_for_analysis, RESULTS_DIR, ANIMAL_ID, DATE_STR)
plt.show()

In [ ]:
# Plot final positions by target type
fig_final = plot_final_positions_by_target(
    trials_for_analysis, 
    min_duration=TRIAL_MIN_DURATION, 
    max_duration=TRIAL_MAX_DURATION,
    results_dir=RESULTS_DIR,
    animal_id=ANIMAL_ID,
    session_date=DATE_STR
)
plt.show()

## 6. Statistical Comparisons

In [ ]:
# Compare left vs right target performance
fig_lr, lr_stats = compare_left_right_performance(
    trials_for_analysis, 
    left_x=-0.7, 
    right_x=0.7,
    results_dir=RESULTS_DIR,
    animal_id=ANIMAL_ID,
    session_date=DATE_STR
)
if fig_lr is not None:
    plt.show()

In [ ]:
# Compare visible vs invisible target performance
fig_vis, vis_stats = compare_visible_invisible_performance(
    trials_for_analysis, 
    results_dir=RESULTS_DIR,
    animal_id=ANIMAL_ID,
    session_date=DATE_STR
)
if fig_vis is not None:
    plt.show()

## 7. Session Summary

In [ ]:
# Calculate summary statistics
valid_trials = [t for t in trials_for_analysis if t.get('has_eye_data', True)]
durations = [t['duration'] for t in valid_trials]
path_lengths = [t['path_length'] for t in valid_trials]
efficiencies = [t['path_efficiency'] for t in valid_trials]

print("="*60)
print("SESSION SUMMARY")
print("="*60)
print(f"Folder: {FOLDER_PATH}")
print(f"Animal: {ANIMAL_ID}")
print(f"Date: {DATE_STR}")
print(f"Valid trials: {len(valid_trials)}")
print(f"\nTime to Target:")
print(f"  Mean: {np.mean(durations):.2f} ± {np.std(durations):.2f} s")
print(f"  Median: {np.median(durations):.2f} s")
print(f"  Range: {np.min(durations):.2f} - {np.max(durations):.2f} s")
print(f"\nPath Length:")
print(f"  Mean: {np.mean(path_lengths):.3f} ± {np.std(path_lengths):.3f}")
print(f"  Median: {np.median(path_lengths):.3f}")
print(f"\nPath Efficiency (1.0 = perfectly direct):")
print(f"  Mean: {np.mean(efficiencies):.3f} ± {np.std(efficiencies):.3f}")
print(f"  Median: {np.median(efficiencies):.3f}")
print("="*60)

In [ ]:
# Create a summary DataFrame for all trials
trial_summary = pd.DataFrame([
    {
        'trial_number': t['trial_number'],
        'duration': t['duration'],
        'target_x': t['target_x'],
        'target_y': t['target_y'],
        'target_visible': t['target_visible'],
        'path_length': t['path_length'],
        'path_efficiency': t['path_efficiency'],
        'trial_failed': t.get('trial_failed', False),
        'has_eye_data': t.get('has_eye_data', True),
    }
    for t in trials_all
])

# Display summary
print("Trial Summary Table:")
display(trial_summary.head(20))

## 8. Fixation Analysis (Interactive)

This section uses the interactive fixation viewer to step through trials one by one.

**Note:** For the interactive viewer to work properly, you may need to use `%matplotlib qt` or `%matplotlib widget` instead of `%matplotlib inline`. The viewer uses keyboard events (press SPACE to advance).

In [ ]:
# Switch to interactive backend for the fixation viewer
# Uncomment one of these if needed:
# %matplotlib qt
# %matplotlib widget

# Run the interactive fixation viewer
# Press SPACE to advance to next trial, ESC or 'q' to quit
print("Starting interactive fixation viewer...")
print("Press SPACE to advance to next trial, ESC or 'q' to quit")
interactive_fixation_viewer(trials_for_analysis, animal_id=ANIMAL_ID, session_date=DATE_STR)

In [ ]:
# Switch back to inline plotting after interactive viewer
%matplotlib inline

## 9. Export Data

In [ ]:
# Save trial summary to CSV
if RESULTS_DIR:
    RESULTS_DIR.mkdir(parents=True, exist_ok=True)
    csv_path = RESULTS_DIR / f"{ANIMAL_ID}_trial_summary.csv"
    trial_summary.to_csv(csv_path, index=False)
    print(f"Saved trial summary to: {csv_path}")